<a href="https://colab.research.google.com/github/ycyuki/VR-project/blob/main/Image_based_Prediction_Modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
# Define the F1 score metric
class F1Score(tf.keras.metrics.Metric):
    def __init__(self, name='f1_score', **kwargs):
        super(F1Score, self).__init__(name=name, **kwargs)
        self.precision = tf.keras.metrics.Precision()
        self.recall = tf.keras.metrics.Recall()

    def update_state(self, y_true, y_pred, sample_weight=None):
        self.precision.update_state(y_true, y_pred, sample_weight)
        self.recall.update_state(y_true, y_pred, sample_weight)

    def result(self):
        p = self.precision.result()
        r = self.recall.result()
        return 2 * ((p * r) / (p + r + tf.keras.backend.epsilon()))

    def reset_states(self):
        self.precision.reset_states()
        self.recall.reset_states()

# Data preprocessing

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!! do not run this cell if the dataset is already create
# Spilt the dataset into train, validation
import os
import pathlib
import random
import shutil
random.seed(42)
# Set the base directory path
new_base_dir = pathlib.Path('/content/drive/MyDrive/793Project/Tower_photo/Newmodel/Train')

# Set the paths for the raw data folders
unstable_dir = new_base_dir / 'Unstable'
stable_dir = new_base_dir / 'Stable'

# Set the paths for the new data folders
data_dir = new_base_dir.parent / 'data'
train_dir = data_dir / 'train'
val_dir = data_dir / 'val'

# Create the new data folders if they don't exist
(train_dir / 'unstable').mkdir(parents=True, exist_ok=True)
(val_dir / 'unstable').mkdir(parents=True, exist_ok=True)
(train_dir / 'stable').mkdir(parents=True, exist_ok=True)
(val_dir / 'stable').mkdir(parents=True, exist_ok=True)

# Function to split the data
def split_data(source_dir, train_dir, val_dir, train_ratio=0.85, val_ratio=0.15):
    files = os.listdir(source_dir)
    if not files:
        print(f"The {source_dir} directory is empty. Skipping...")
        return

    random.shuffle(files)
    num_files = len(files)
    train_count = int(num_files * train_ratio)
    val_count = int(num_files * val_ratio)

    for i, file in enumerate(files):
        src_path = source_dir / file

        if i < train_count:
            dest_dir = train_dir
        elif i < train_count + val_count:
            dest_dir = val_dir
        else:
            dest_dir = test_dir

        dest_path = dest_dir / file
        shutil.copy(src_path, dest_path)

# Split the unstable images
split_data(unstable_dir, train_dir / 'unstable', val_dir / 'unstable')

# Split the stable images
split_data(stable_dir, train_dir / 'stable', val_dir / 'stable')

In [ ]:
from pathlib import Path
# Count the number of files in each subdirectory
data_dir = Path('/content/drive/MyDrive/793Project/Tower_photo/Newmodel/data')

for folder in data_dir.iterdir():
    print(f"Directory: {folder.name}")
    for sub_folder in folder.iterdir():
        file_count = len(list(sub_folder.iterdir()))
        print(f"  Subdirectory: {sub_folder.name}, Number of files: {file_count}")

In [5]:
# Set the base directory path
data_dir = Path('/content/drive/MyDrive/793Project/Tower_photo/Newmodel/data')
# Set the paths for the new data folders
train_dir = data_dir / 'train'
val_dir = data_dir / 'val'
test_dir = Path('/content/drive/MyDrive/793Project/Tower_photo/Newmodel/Test')

# Load the datasets
train_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    shuffle=True,
    image_size=(644, 394),
    batch_size=32
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    val_dir,
    shuffle=True,
    image_size=(644, 394),
    batch_size=32
)

test_ds = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    shuffle=True,
    image_size=(644, 394),
    batch_size=32
)

Found 224 files belonging to 2 classes.
Found 38 files belonging to 2 classes.
Found 44 files belonging to 2 classes.


# Use ResNet50 pre-train model

In [6]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model


In [7]:
# Load the ResNet50 model
base_model = ResNet50(weights='imagenet', include_top=False)

94765736/94765736 [==============================] - 1s 0us/step


In [ ]:
# Freeze the layers of the base_model
for layer in base_model.layers:
    layer.trainable = False

# Add custom layers on top
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Print the model summary
model.summary()

In [9]:
# Compile the model
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), F1Score()])

In [ ]:
# Train the model
history = model.fit(
    train_ds,
    epochs=10,
    validation_data=val_ds
)

In [ ]:
from matplotlib import pyplot as plt
# Plotting training and validation loss
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

# Plotting training and validation accuracy
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Evaluate the model on the test dataset
model.evaluate(test_ds)

In [ ]:
# Load the test dataset without shuffling for prediction clarity
test_ds = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    shuffle=False,  # Disable shuffle for prediction phase
    image_size=(644, 394),
    batch_size=32
)

# Predict with the model
predictions = model.predict(test_ds)

# Get the predicted class labels with the highest probability
predicted_labels = np.argmax(predictions, axis=1)

# Get the maximum probability for each prediction (confidence of prediction)
predicted_probabilities = np.max(predictions, axis=1)

# Evaluate the model's performance on the test dataset
evaluation = model.evaluate(test_ds)

# Print the results for each image with their file paths
file_paths = test_ds.file_paths
for i, (prob, label) in enumerate(zip(predicted_probabilities, predicted_labels)):
    print(f"Image {i} ({file_paths[i]}): Predicted Label = {label}, Probability = {prob:.4f}")

# Print detailed probabilities for each class per image
for i, prediction in enumerate(predictions):
    print(f"Image {i} ({file_paths[i]}):")
    for label_index, prob in enumerate(prediction):
        print(f"  Label {label_index}: Probability = {prob:.4f}")

# Print a sample of file paths
print("Sample file paths:", file_paths[:5])


# Grad-CAM heatmap for ResNet50 model

In [15]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions, ResNet50
from tensorflow.keras.preprocessing import image
import cv2
import matplotlib.pyplot as plt
import matplotlib.cm as cm

In [16]:
# Function to generate GradCAM heatmap
def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    grad_model = tf.keras.models.Model(
        inputs=[model.inputs],
        outputs=[model.get_layer(last_conv_layer_name).output, model.output]
    )

    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        print("Conv outputs shape:", conv_outputs.shape)
        print("Predictions shape:", predictions.shape)

        if pred_index is None:
            predictions = tf.keras.layers.GlobalAveragePooling2D()(predictions)
            pred_index = tf.argmax(predictions[0])
            pred_index = int(pred_index.numpy())

        class_channel = predictions[0, pred_index]

        grads = tape.gradient(class_channel, conv_outputs)
        pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
        conv_outputs = conv_outputs[0]
        heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
        heatmap = tf.squeeze(heatmap)
        heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

# Function to process image
def process_image(img_path, size=(644, 394)):
    img = image.load_img(img_path, target_size=size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    return preprocess_input(img_array)

In [ ]:
# Image paths for testing
img_paths = ['/content/drive/MyDrive/793Project/Tower_photo/Newmodel/Test/Stable/Tower_1_Rotation_22.5_2024-04-03_13-02-43.png', # prediction: stable
             '/content/drive/MyDrive/793Project/Tower_photo/Newmodel/Test/Stable/Tower_47_Rotation_135_2024-04-03_13-30-14.png', # prediction: stable
             '/content/drive/MyDrive/793Project/Tower_photo/Newmodel/Test/Stable/Tower_47_Rotation_157.5_2024-04-03_13-30-15.png', # prediction: stable
             '/content/drive/MyDrive/793Project/Tower_photo/Newmodel/Test/Stable/Tower_47_Rotation_315_2024-04-03_13-30-19.png', # prediction: unstable
             '/content/drive/MyDrive/793Project/Tower_photo/Newmodel/Test/Unstable/Tower_13_Rotation_135_2024-04-03_13-09-14.png', # prediction: unstable
             '/content/drive/MyDrive/793Project/Tower_photo/Newmodel/Test/Unstable/Tower_71_Rotation_135_2024-04-03_13-40-58.png', # prediction: stable
             '/content/drive/MyDrive/793Project/Tower_photo/Newmodel/Test/Unstable/Tower_71_Rotation_225_2024-04-03_13-41-00.png', # prediction: unstable
             '/content/drive/MyDrive/793Project/Tower_photo/Newmodel/Test/Unstable/Tower_71_Rotation_315_2024-04-03_13-41-01.png', # prediction: stable
             '/content/drive/MyDrive/793Project/Tower_photo/Newmodel/Test/Unstable/Tower_71_Rotation_45_2024-04-03_13-40-56.png', # prediction: stable
             '/content/drive/MyDrive/793Project/Tower_photo/Newmodel/Test/Unstable/Tower_13_Rotation_315_2024-04-03_13-09-18.png' # prediction: unstable
             ]

# Model details
last_conv_layer_name = 'conv5_block2_out' # Name of the convolutional layer that will be used for GradCAM
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(644, 394, 3))
resnet50_model_gradcam = base_model


# Process each image and compute Grad-CAM heatmap
for img_path in img_paths:
    img_array = process_image(img_path)
    heatmap = make_gradcam_heatmap(img_array, resnet50_model_gradcam, last_conv_layer_name)

    # Read and prepare the image
    color_img = cv2.imread(img_path)
    color_img = cv2.cvtColor(color_img, cv2.COLOR_BGR2RGB)
    img = cv2.cvtColor(color_img, cv2.COLOR_RGB2GRAY)
    img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)

    # Rescale heatmap to a range 0-255 and apply colormap
    heatmap = np.uint8(255 * heatmap)
    colored_heatmap = cm.jet(heatmap)[:,:,:3]
    colored_heatmap = cv2.resize(colored_heatmap, (img.shape[1], img.shape[0]))
    colored_heatmap = np.uint8(255 * colored_heatmap)

    # Superimpose the heatmap on the original image
    alpha = 0.6  # Heatmap transparency
    superimposed_img = colored_heatmap * alpha + img * (1 - alpha)
    superimposed_img = np.clip(superimposed_img, 0, 255)

    # Set up Matplotlib figure and axes
    fig, ax = plt.subplots(1, 2, figsize=(12, 6))

    # Display original image
    ax[0].imshow(color_img)
    ax[0].axis('off')
    ax[0].set_title('Original Image')

    # Display the image with the superimposed heatmap
    ax[1].imshow(superimposed_img.astype('uint8'))
    ax[1].axis('off')
    ax[1].set_title('Image with Grad-CAM')

    plt.show()

# Use InceptionV3 pre-train model

In [18]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image

In [ ]:
# Function to preprocess the dataset
def preprocess_dataset(dataset):
    resize_and_rescale = tf.keras.Sequential([
        tf.keras.layers.experimental.preprocessing.Resizing(299, 299),
        tf.keras.layers.experimental.preprocessing.Rescaling(1./255)
    ])

    def apply_preprocessing(x, y):
        return resize_and_rescale(x), y

    return dataset.map(apply_preprocessing)

# Preprocess the datasets
train_ds = preprocess_dataset(train_ds)
val_ds = preprocess_dataset(val_ds)
test_ds = preprocess_dataset(test_ds)

# Configure datasets for performance
AUTOTUNE = tf.data.AUTOTUNE
i_train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
i_val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
i_test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

# Load InceptionV3 model pre-trained on ImageNet data
base_model = tf.keras.applications.InceptionV3(include_top=False,
                                               weights='imagenet',
                                               input_shape=(299, 299, 3))

# Freeze the base model
base_model.trainable = False

# Create a new model on top of the base model
inputs = tf.keras.Input(shape=(299, 299, 3))
x = base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.2)(x)  # Regularize with dropout
outputs = layers.Dense(1, activation='sigmoid')(x)
i_model = tf.keras.Model(inputs, outputs)

In [ ]:
# Print model summary
i_model.summary()

In [23]:
# Compile the model
i_model.compile(optimizer='adam',
                loss='binary_crossentropy',
                metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), F1Score()])


In [ ]:
# Train the model
history = i_model.fit(i_train_ds,
                    epochs=10,
                    validation_data=i_val_ds)

In [ ]:
# Plotting training and validation loss
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

# Plotting training and validation accuracy
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Evaluate the model
i_model.evaluate(i_test_ds)

In [ ]:
# Predict the test dataset
predictions = i_model.predict(i_test_ds)

# Get the predicted class labels with the highest probability
predicted_labels = np.argmax(predictions, axis=1)

# Get the maximum probability for each prediction (confidence of prediction)
predicted_probabilities = np.max(predictions, axis=1)

# Evaluate the model's performance on the test dataset overall
evaluation = i_model.evaluate(i_test_ds)

# Print evaluation metrics
print(f"Model evaluation: {evaluation}")

In [ ]:
# Image paths for testing
img_paths = [
    '/content/drive/MyDrive/793Project/Tower_photo/Newmodel/Test/Stable/Tower_1_Rotation_0_2024-04-03_13-02-42.png',
    '/content/drive/MyDrive/793Project/Tower_photo/Newmodel/Test/Stable/Tower_1_Rotation_112.5_2024-04-03_13-02-45.png',
     '/content/drive/MyDrive/793Project/Tower_photo/Newmodel/Test/Stable/Tower_47_Rotation_0_2024-04-03_13-30-11.png',
    '/content/drive/MyDrive/793Project/Tower_photo/Newmodel/Test/Stable/Tower_47_Rotation_112.5_2024-04-03_13-30-14.png',
    '/content/drive/MyDrive/793Project/Tower_photo/Newmodel/Test/Unstable/Tower_93_Rotation_135_2024-04-03_13-49-05.png',
    '/content/drive/MyDrive/793Project/Tower_photo/Newmodel/Test/Unstable/Tower_93_Rotation_225_2024-04-03_13-49-07.png'
]

# Function to load and preprocess image
def load_and_preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(299, 299))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    return preprocess_input(img_array)

# Load and preprocess all images
images = np.vstack([load_and_preprocess_image(img_path) for img_path in img_paths])

# Predict on individual images
predictions = i_model.predict(images)

# Get predicted labels and probabilities for individual images
predicted_labels = np.argmax(predictions, axis=1)
predicted_probabilities = np.max(predictions, axis=1)

# Print predictions for each image
for i, (prob, label) in enumerate(zip(predicted_probabilities, predicted_labels)):
    print(f"Image {i}: Predicted Label = {label}, Probability = {prob:.4f}")


# GradCAM heatmap for InceptionV3 model

In [29]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image
import matplotlib.cm as cm
import cv2
import matplotlib.pyplot as plt

In [30]:
# Define Grad-CAM function
def make_gradcam_heatmap(img_array, base_model, last_conv_layer, pred_index=None):
    # Create a new Grad-CAM model
    grad_model = tf.keras.models.Model(
        inputs=base_model.inputs,
        outputs=[last_conv_layer.output, base_model.output]
    )

    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0], axis=-1)
        pred_index_flat = tf.reshape(pred_index, [-1])
        one_hot = tf.one_hot(pred_index_flat, depth=preds.shape[-1])
        preds = tf.reshape(preds, (-1, preds.shape[-1]))
        class_channel = tf.matmul(preds, one_hot, transpose_b=True)
        grads = tape.gradient(class_channel, last_conv_layer_output)
        pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
        last_conv_layer_output = last_conv_layer_output[0]
        heatmap = tf.reduce_sum(last_conv_layer_output * tf.expand_dims(tf.expand_dims(pooled_grads, axis=0), axis=0), axis=-1)
        heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

In [ ]:
# Load and preprocess image
img_path = '/content/drive/MyDrive/793Project/Tower_photo/Newmodel/Test/Stable/Tower_1_Rotation_0_2024-04-03_13-02-42.png'
img = image.load_img(img_path, target_size=(299, 299))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = preprocess_input(img_array)

# Load base model
base_model = tf.keras.applications.InceptionV3(include_top=False, weights='imagenet', input_shape=(299, 299, 3))
last_conv_layer = base_model.get_layer('mixed10')

# Generate Grad-CAM heatmap
heatmap = make_gradcam_heatmap(img_array, base_model, last_conv_layer)

# Load the original image using cv2
img = cv2.imread(img_path)

# Rescale heatmap to a range 0-255
heatmap = np.uint8(255 * heatmap)

# Use jet colormap to colorize heatmap
jet = cm.get_cmap("jet")

# Use RGB values of the colormap
jet_colors = jet(np.arange(256))[:, :3]
jet_heatmap = jet_colors[heatmap]

# Create an image with RGB colorized heatmap
jet_heatmap = cv2.resize(jet_heatmap, (img.shape[1], img.shape[0]))
jet_heatmap = np.uint8(255 * jet_heatmap)
superimposed_img = jet_heatmap * 0.4 + img

# Display the heatmap
plt.imshow(superimposed_img / 255)
plt.axis('off')
plt.title('Grad-CAM Heatmap')
plt.show()